## Introduction

### Pré-requis

* Ce notebook permet de peupler l'ontologie relative aux adresses définies dans le fichier lié à la variable `ont_file`.
* Avant de procéder à l'exécution du script, il faut installer deux logiciels qui devront être tournés durant le processus :
    * [GraphDB](https://graphdb.ontotext.com/) qui permet de stocker et de travailler sur des graphes de connaissance. Une variable est associée au logiciel : `graphdb_url` qui est l'URL de l'application web. Le procédé est similaire à celui de `ontorefine_url` pour trouver sa valeur.

### Fichiers du dossier
* `ont_file` est le fichier comprenant l'ontologie des adresses.
* `data_file` : le but du notebook est de peupler l'ontologie. Ce peuplement se fait grâce à des exemples d'adresses qu'on trouve dans les annuaires du commerce de la ville de Paris dont les adresses sont stockées localement dans le fichier json lié à la variable `data_file`.

### Présentation du processus
* Le processus est divisé en quatre parties :
    * la première fait des traitements généraux :
        * import des librairies
        * définition des variables et modification de ces dernières si nécessaire (conversion des chemins relatifs en chemins absolus pour éviter les mauvaises surprises par exemple)
        * créer de dossiers s'ils n'existent pas (`out_folder` ou `tmp_folder`).
    * la deuxième est liée à la création des données à partir du fichier `data_file` :
        * pour chaque adresse définie dans `data_file`, on la structure selon l'ontologie définie et on l'importe dans un graphe provisoire (`Graph()` de `rdflib`) ;
        * export des données du graphe dans le fichier `out_file` ;
    * la troisième est liée à l'import des données présentes dans `data_file` dans GraphDB :
        * création d'un répertoire ayant `project_name` comme identifiant. S'il existe déjà, on le vide de tous ses triplets. C'est pourquoi il faut éviter de définir un identifiant qui se réfère à un répertoire déjà existant ;
        * import de l'ontologie présente dans `ont_file` et des données créées dans la section précédente qui sont dans `out_file` ;
        * :warning: pas encore traité ici : nettoyage des données grâce aux requêtes présentes dans `query_file`.
    * la dernière partie permet de répondre aux questions de compétence liées au modelet adresse qui a mené à la création du fichier `ont_file`. Les réponses à ces questions sont enregistrés dans le dossier `out_folder` sous la forme des fichiers csv. Les questions sont les suivantes :
        * *Quelles sont les adresses faisant référence à un lieu dont la localisation est située dans une zone géographique donnée ?*
        * *Quelles sont les adresses faisant référence à un numéro d'immeuble situé le long d'une rue donnée ?*
        * *Quelles sont les coordonnées d'une adresse donnée (définie par son libellé) ?*

### Structure du fichier d'entrée
Le fichier d'entrée `data_file` est un fichier `json` qui doit respecter une certaine syntaxe.
* Il y doit avoir une clé `addresses` qui doit être associée à une liste de dictionnaires décrivrant chacun une adresse : `{"addresses":[addr1, addr2, ..., addrN]}`
* Comme dit précédemment, chaque addresse est un dictionnaire et doit comprendre les clés-valeurs suivantes :
    * `label` / `{label de l'adresse}` qui permet de décrire le libellé de l'adresse ;
    * `date` / `{date de validité de l'adresse}` décrit la date à laquelle l'adresse est valable, la valeur doit respecter le format `yyyy-MM-ddTHH:mm:ss` ;
    * `lang` / `{code de la langue}` décrit la langue dans laquelle est exprimée l'adresse, `null` est la valeur par défaut ;
    * `target` / `{réference de la cible}` : décrit le repère cible de l'adresse. Dans la structure présente, chaque repère est défini par un identifiant unique qu'on appelle référence. Si l'adresse cible un repère connu par un identifiant, la valeur associée à `target` vaut la référence, sinon la valeur est `null`.
    * `landmarks` / `{dictionnaire de repères}` : cet élément stocke l'ensemble des repères détectés dans le libellé de l'adresse dans un dictionnaire, chaque clé définit un repère de manière unique au sein de l'adresse. Un repère est décrit par l'intermédiaire d'un dictionnaire qui contient au minimum :
        * `label` / `{label du repère}` qui permet de décrire le libellé du repère au sein de l'adresse ;
        * `type` / `{type du repère}` qui permet de décrire le type du repère (`housenumber`, `thoroughfare`, `district`, `city`, `structure`, `undefined`)
    * `spatial_relations` / `{liste des relations spatiales}` décrit l'ensemble des relations spatiales entre les repères décrites dans l'adresse (explicite ou non). Chaque relation est représentée par une liste des 3 éléments :
		* libellé de la relation spatiale
		* référence du repère ayant le rôle de locatum ; 
		* liste des références des repères ayant le rôle de relatum (:warning: même s'il n'y a qu'un repère, il doit être stocké dans une liste) 

Ici, voici l'exemple d'un fichier json contenant une seule adresse :
```json
{
   "addresses":[
      {
         "label":"situé sur le blv. de Clichy, dans la partie comprise entre la place Blanche et la rue Fontaine",
         "date":"1850-01-01T00:00:00",
         "lang":"fr",
         "target":null,
         "landmarks":{
            "bd_clichy":{
               "label":"blv. de Clichy",
               "type":"thoroughfare"
            },
            "pl_blanche":{
               "label":"place Blanche",
               "type":"thoroughfare"
            },
            "rue_fontaine":{
               "label":"rue Fontaine",
               "type":"thoroughfare"
            }
         },
         "spatial_relations":[
            ["sur", null, ["bd_clichy"]],
            ["dans la partie comprise entre", null, ["pl_blanche", "rue_fontaine"]]
         ]
      }
   ]
}
```

## 1. Traitements généraux

### Import des librairies

In [ ]:
import os
import sys
from rdflib import Graph

### Import des variables globales

In [ ]:
ont_file_name = "address_ont.ttl"
data_file_name = "adresses.json"
query_file_name = "query.txt"
out_file_name = "addresses.ttl"
local_config_file_name = "repo_config.ttl"

out_folder_name = "out"
tmp_folder_name = "tmp"

project_name = "paris_directories"
graphdb_url = "http://localhost:7200"
local_uri = "http://rdf.geohistoricaldata.org/address#"

py_code_folder_path = "../../code"

### Traitement des variables globales

* Obtention des chemins absolus des fichiers à partir des chemins relatifs donnés dans la section précédente
* Création d'un dossier de sortie s'il n'existe pas encore pour stocker les fichiers de sortie



In [ ]:
tmp_folder = os.path.abspath(tmp_folder_name)
out_folder = os.path.abspath(out_folder_name)
python_code_folder = os.path.abspath(py_code_folder_path)

ont_file = os.path.abspath(ont_file_name)
data_file = os.path.abspath(data_file_name)
query_file = os.path.abspath(query_file_name)
out_file = os.path.join(out_folder, out_file_name)
local_config_file = os.path.join(tmp_folder, local_config_file_name)

### Import des modules situés dans le dossier `code`

In [ ]:
# Appel du dossier `code` comprend les codes python
sys.path.insert(1, python_code_folder)

import graphdb as gd
import graphrdf as gr
import filemanagement as fm

### Création de dossiers s'ils n'existent pas

In [ ]:
fm.create_folder_if_not_exists(out_folder)
fm.create_folder_if_not_exists(tmp_folder)

## 2. Création d'adresses via le fichier JSON

In [ ]:
data = fm.read_json_file(data_file)
g = Graph()

for addr in data.get("addresses"):
    try:
        gr.create_address(g, local_uri, addr.get("label"), addr.get("lang"), addr.get("landmarks"), addr.get("spatial_relations"), addr.get("target"), addr.get("date"))
    except TypeError:
        pass

g.serialize(destination=out_file)

## 3. Import des données dans GraphDB et nettoyage

### Création du répertoire local dans GraphDB
Pour que la création marche, il faut que GraphDB soit lancé et donc que l'URI donné par `graphdb_url` fonctionne. Si le répertoire existe déjà, rien n'est fait

In [ ]:
gd.create_config_local_repository_file(local_config_file, project_name)
gd.create_repository_from_config_file(graphdb_url, local_config_file)

### Vidage du répertoire local
Le répertoire dont l'id est `project_name` pour y stocker les données récupérées, cela est utile si le répertoire existait déjà.

In [ ]:
gd.clear_repository(graphdb_url, project_name)

### Import de l'ontologie et des triplets créés

In [ ]:
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, ont_file)
gd.import_ttl_file_in_graphdb(graphdb_url, project_name, out_file)

## 4. Réponses aux questions de compétence

### Quelles sont les adresses faisant référence à un lieu dont la localisation est située dans une zone géographique donnée ?

In [ ]:
proj_uri = "http://www.opengis.net/def/crs/OGC/1.3/CRS84"
polygon_wkt = "POLYGON ((2.338886 48.858207, 2.345817 48.857007, 2.352061 48.854847, 2.359893 48.851727, 2.360559 48.851458, 2.360601 48.850061, 2.360601 48.849114, 2.356503 48.850216, 2.352576 48.851388, 2.349722 48.852094, 2.348199 48.852743, 2.344787 48.854042, 2.342191 48.855285, 2.33968 48.857134, 2.338328 48.857882, 2.338886 48.858207))" #Polygon describes area of Centre Paris islands
out_file_name = "query1-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>
PREFIX gspf: <http://www.opengis.net/def/function/geosparql/>

select ?s ?nb ?streetName ?coords where {{
    ?s a addr:Address; addr:targets [rdfs:label ?nb;
                       addr:isPartOf [addr:isLandmarkType addr:Thoroughfare; rdfs:label ?streetName];
                                                                              gsp:asWKT ?coords].
        FILTER (
        gspf:sfWithin(
            ?coords,
            "<{proj_uri}> {polygon_wkt}"^^gsp:wktLiteral
        )
    )

}}
"""

gd.select_query_to_txt_file(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))

### Quelles sont les adresses ayant référence un lien avec une rue donnée ?

In [ ]:
street_name = "place Blanche"
lang = "fr"
out_file_name = "query2-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
SELECT ?address ?addressLabel ?geom WHERE {{
    ?lm a addr:Landmark; addr:isLandmarkType addr:Thoroughfare; rdfs:label ?streetName.
    FILTER(LCASE(STR(?streetName)) = \"{street_name.lower()}\" && LANG(?streetName) = \"{lang}\")
    ?address a addr:Address; rdfs:label ?addressLabel.
    OPTIONAL {{?address addr:targets [geo:asWKT ?geom].}}
    {{?address addr:firstStep ?addrSegment}}UNION{{?address addr:firstStep [addr:nextStep+ ?addrSegment]}}
    ?addrSegment a addr:AddressSegment; addr:relatum ?lm.
}}
"""

gd.select_query_to_txt_file(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))

### Quelles sont les coordonnées d'une adresse donnée (définie par son libellé) ?

In [ ]:
address_name = "3 rue de la vieille place aux veaux, Paris"
lang = "fr"
out_file_name = "query3-out.csv"

query = f"""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>
PREFIX gsp: <http://www.opengis.net/ont/geosparql#>

SELECT ?item ?addressLabel ?coords WHERE {{
    ?item a addr:Address;
          rdfs:label ?addressLabel.
    OPTIONAL {{ ?item addr:targets [gsp:asWKT ?coords]. }}
    FILTER(LCASE(STR(?addressLabel)) = \"{address_name.lower()}\" && LANG(?addressLabel) = \"{lang}\")
}}
"""

gd.select_query_to_txt_file(query, graphdb_url, project_name, os.path.join(out_folder,out_file_name))